In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


repo_id = "./ckpt/llama-2-13b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(repo_id, device_map='auto', torch_dtype=torch.float16, load_in_8bit=False)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(repo_id, use_fast=False)


[2023-09-11 16:17:49,748] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 3/3 [00:52<00:00, 17.59s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [3]:
tokenizer.pad_token = tokenizer.eos_token
input_ids = tokenizer(['<s>Human: Introduce the history of Shenzhen\n</s><s>Assistant: '], return_tensors="pt",add_special_tokens=False).input_ids.to('cuda')        
generate_input = {
    "input_ids":input_ids,
    "max_new_tokens":512,
    "do_sample":True,
    "top_k":50,
    "top_p":0.95,
    "temperature":0.3,
    "repetition_penalty":1.3,
    "eos_token_id":tokenizer.eos_token_id,
    "bos_token_id":tokenizer.bos_token_id,
    "pad_token_id":tokenizer.pad_token_id
}
generate_ids  = model.generate(**generate_input)
text = tokenizer.decode(generate_ids[0])
print(text)

<s> Human: Introduce the history of Shenzhen
</s><s> Assistant:  Sure, I'd be happy to help! Here is a brief overview of the history of Shenzhen:

Shenzhen has undergone one of the most rapid transformations in modern Chinese history. Prior to 1978, it was just a small fishing village with less than 20,000 residents. However, after China began its economic reforms and opened up to foreign investment, Shenzhen became one of the first Special Economic Zones (SEZs) established by the government. This designation allowed for greater autonomy from central planning and regulations, which attracted many domestic and international businesses looking to take advantage of cheap labor costs and favorable policies. As a result, Shenzhen experienced an influx of migrant workers who came seeking better job opportunities; today there are now more than 25 million people living within city limits making it one of fastest growing cities globally not only economically but also demographically speaking to

In [4]:
from bigmodelvis import Visualization

Visualization(model).structure_graph();

root
├── model (LlamaModel)
│   ├── embed_tokens (Embedding) weight:[32000, 5120]
│   ├── layers (ModuleList)
│   │   └── 0-39(LlamaDecoderLayer)
│   │       ├── self_attn (LlamaAttention)
│   │       │   └── q_proj,k_proj,v_proj,o_proj(Linear) weight:[5120, 5120]
│   │       ├── mlp (LlamaMLP)
│   │       │   ├── gate_proj,up_proj(Linear) weight:[13824, 5120]
│   │       │   └── down_proj (Linear) weight:[5120, 13824]
│   │       └── input_layernorm,post_attention_layernorm(LlamaRMSNorm) weight:[5120]
│   └── norm (LlamaRMSNorm) weight:[5120]
└── lm_head (Linear) weight:[32000, 5120]